In [ ]:
import numpy as np
import json
from PIL import Image, ImageDraw
import os
import cv2
import pandas as pd
from tqdm import tqdm
import shutil
import random

The purpose of this script is to explore images/annotations of the WIDER dataset.  
Also it converts annotations into json format.

In [ ]:
# first run this script for this images:
IMAGES_DIR = '/home/gpu2/hdd/dan/WIDER/WIDER_train/images/'
BOXES_PATH = '/home/gpu2/hdd/dan/WIDER/wider_face_split/wider_face_train_bbx_gt.txt'
RESULT_DIR = '/home/gpu2/hdd/dan/WIDER/train/'

# then run for this images:
# IMAGES_DIR = '/home/gpu2/hdd/dan/WIDER/WIDER_val/images/'
# BOXES_PATH = '/home/gpu2/hdd/dan/WIDER/wider_face_split/wider_face_val_bbx_gt.txt'
# RESULT_DIR = '/home/gpu2/hdd/dan/WIDER/train_part2/'

# Read data

In [ ]:
# collect paths to all images

all_paths = []
for path, subdirs, files in tqdm(os.walk(IMAGES_DIR)):
    for name in files:
        all_paths.append(os.path.join(path, name))
        
metadata = pd.DataFrame(all_paths, columns=['full_path'])

# strip root folder
metadata['path'] = metadata.full_path.apply(lambda x: os.path.relpath(x, IMAGES_DIR))

In [ ]:
# see all unique endings
metadata.path.apply(lambda x: x.split('.')[-1]).unique()

In [ ]:
# number of images
len(metadata)

In [ ]:
# read annotations
with open(BOXES_PATH, 'r') as f:
    content = f.readlines()
    content = [s.strip() for s in content]

In [ ]:
# split annotations by image
boxes = {}
num_lines = len(content)
i = 0
name = None

while i < num_lines:
    s = content[i]
    if s.endswith('.jpg'):
        if name is not None:
            assert len(boxes[name]) == num_boxes
        name = s
        boxes[name] = []
        i += 1
        num_boxes = int(content[i])
        i += 1
    else:
        xmin, ymin, w, h = s.split(' ')[:4]
        xmin, ymin, w, h = int(xmin), int(ymin), int(w), int(h)
        if h <= 0 or w <= 0:
            print(name)  
            # some boxes are weird!
            # so i don't use them
            num_boxes -= 1
        else:
            boxes[name].append((xmin, ymin, w, h))
        i += 1

In [ ]:
# check that all images have bounding boxes
assert metadata.path.apply(lambda x: x in boxes).all()

# Show some bounding boxes

In [ ]:
def draw_boxes_on_image(path, boxes):

    image = Image.open(path)
    draw = ImageDraw.Draw(image, 'RGBA')
    width, height = image.size

    for b in boxes:
        xmin, ymin, w, h = b
        xmax, ymax = xmin + w, ymin + h

        fill = (255, 255, 255, 45)
        outline = 'red'
        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
            fill=fill, outline=outline
        )
    return image

In [ ]:
i = random.randint(0, len(metadata) - 1)  # choose a random image
some_boxes = boxes[metadata.path[i]]
draw_boxes_on_image(metadata.full_path[i], some_boxes)

# Convert

In [ ]:
def get_annotation(path, width, height):
    name = path.split('/')[-1]
    annotation = {
      "filename": name,
      "size": {"depth": 3, "width": width, "height": height}
    }
    objects = []
    for b in boxes[path]:
        xmin, ymin, w, h = b
        xmax, ymax = xmin + w, ymin + h
        objects.append({
            "bndbox": {"ymin": ymin, "ymax": ymax, "xmax": xmax, "xmin": xmin}, 
            "name": "face"
        })
    annotation["object"] = objects
    return annotation

In [ ]:
# create a folder for the converted dataset
shutil.rmtree(RESULT_DIR, ignore_errors=True)
os.mkdir(RESULT_DIR)
os.mkdir(os.path.join(RESULT_DIR, 'images'))
os.mkdir(os.path.join(RESULT_DIR, 'annotations'))

In [ ]:
for T in tqdm(metadata.itertuples()):
    
    # get width and height of an image
    image = cv2.imread(T.full_path)
    h, w, c = image.shape
    assert c == 3
    
    # name of the image
    name = T.path.split('/')[-1]
    assert name.endswith('.jpg')

    # copy the image
    shutil.copy(T.full_path, os.path.join(RESULT_DIR, 'images', name))
    
    # save annotation for it
    d = get_annotation(T.path, w, h)
    json_name = name[:-4] + '.json'
    json.dump(d, open(os.path.join(RESULT_DIR, 'annotations', json_name), 'w')) 